In [2]:
%plot inline -w 640

ErrorException: syntax: "%" is not a unary operator

# Nonlinear optimization - 2022.2

## Optimum power for a set of $m$ lamps illuminating a set of $n$ flat patches to best approach a target illumination

Consider $m$ lamps illuminating $n$ (small flat) patches. The illumination intensity $I_k$ at the $k$-th patch depends linearly on the lamp powers $p_j$ as:
$$
I_k = \sum\limits_{j=1}^m a_{k,j}p_j, \quad \text{with} \quad a_{k,j} = \dfrac{\max\{\cos(\theta_{k,j}), 0\}}{r^{2}_{k,j}},
$$
where $r_{k,j}$ is the length of the vector $\mathbf{r}_{k,j}$ connecting the center of the $k$-th patch to the position of the $j$-th lamp and $\theta_{k,j}$ is the angle between the patch normal vector $\mathbf{n}_{k}$ and $\mathbf{r}_{k,j}$. See the Convex Optimization book slides for more details.

The proposed problem is to achieve a desired illumination $I_{des}$ with bounded lamp powers ($p_{max}$), i.e.,
$$
\begin{align*}
    & \min. \underset{k=1, 2, \ldots, n}{\max}\vert \log(I_k) - \log(I_{des})\vert \\
    \text{s. t.}\quad & 0 \leq p_j \leq p_{max}, j = 1, 2, \ldots, m.
\end{align*}
$$


### Suboptimally solve the problem using, e.g., Matlab, according to the following approaches:

1. Using uniform power, i.e., $p_j = p, 0 \leq p \leq p_{max}$.
2. Using least-squares, i.e., $ \min. \sum\limits_{k=1}^n (I_k - I_{des})^2$, and rounding $p_j$ as $p_j = \max\{0, \min\{p_j, p_{max}\}\}$.
3. Using weighted least-squares, i.e., $ \min. \sum\limits_{k=1}^n (I_k - I_{des})^2 + \sum\limits_{k=1}^n w_j (p_j - p_{max})^2$ and iteratively adjusting the weights $w_j$ until $0 \leq p \leq p_{max}, \forall j$.
4. Using linear programming, i.e., 
$$
\begin{align*}
    & \min. \underset{k=1, 2, \ldots, n}{\max}\vert I_k - I_{des}\vert \\
    \text{s. t.}\quad & 0 \leq p_j \leq p_{max}, j = 1, 2, \ldots, m.
\end{align*}
$$

### Solve the problem optimally using convex optimization

For this goal, consider the equivalent convex problem
$$
\begin{align*}
    & \min. \underset{k=1, 2, \ldots, n}{\max}h(I_k/I_{des}) \\
    \text{s. t.}\quad & 0 \leq p_j \leq p_{max}, j = 1, 2, \ldots, m,
\end{align*}
$$
where $h(u) = \max\{u, 1/u\}$.


In [3]:
clc;
close all;

showFig = true;
numLamps = 4;
numPatches = 5;
maxPwr = 1;
desiredillum = 0.01;

% Code for random generation
% rng(20222);
% lampPos = repmat([0.5 6], numLamps, 1) + 3*rand(numLamps, 2);
% lampPos(1:end, 1) = cumsum(lampPos(:, 1));
% patchPos = repmat([0 2], numPatches+1, 1) + 2*rand(numPatches+1, 2);
% patchPos(1:end, 1) = cumsum(patchPos(:, 1));
% lampPos(:, 1) = lampPos(:, 1) + patchPos(1, 1);

lampPos = [0.6838 7.8976; 3.1835 7.0690; 5.5327 6.6806; 6.1963 6.5203];
patchPos = [0.0762 2.8621; 1.4506 2.8733; 2.5122 2.5864; 4.2246 3.5846; 5.3512 2.1960; 6.3013 2.9838];

lampPos = complex(lampPos(:,1), lampPos(:,2));
patchPos = complex(patchPos(:,1), patchPos(:,2));
patchCenter = (patchPos(1:end-1) + patchPos(2:end))/2;
patchNorm = patchPos(2:end) - patchPos(1:end-1);
minPatchLength = min(abs(patchNorm))/2;
for p = 1:length(patchNorm)
    n = null([real(patchNorm(p)) imag(patchNorm(p))]);
    patchNorm(p) = minPatchLength*complex(n(1), n(2));
end

if showFig == true
    plot(lampPos, 'ro', 'MarkerFaceColor', 'auto');
    hold on;
    plot(patchPos, 'bo-', 'MarkerFaceColor', 'auto');
    plot(patchCenter, 'mx', 'MarkerFaceColor', 'auto');
    plot(patchNorm + patchCenter, 'kd', 'MarkerFaceColor', 'auto');
    for i=1:length(patchCenter),
        plot([patchCenter(i) patchNorm(i) + patchCenter(i)], 'k--', 'MarkerFaceColor', 'auto');
    end
    axis equal;
end
ylabel('y');
xlabel('x');

radDist = zeros(length(patchCenter), length(lampPos));
angShift = zeros(length(patchCenter), length(lampPos));
for i = 1:length(patchCenter)
    for j = 1:length(lampPos)
        radDist(i, j) = abs(patchCenter(i) - lampPos(j));
        angShift(i, j) = angle(lampPos(j)-patchCenter(i))-angle(patchNorm(i));
    end
end

illumCoeff = radDist.^(-2).*max(cos(angShift), 0);

UndefVarError: UndefVarError: clc not defined

### 1. Using uniform power

For the first method, we have that $p_j = p$ for $j \in \{1, 2, \cdots, m\}$. Therefore, the optimization problem becomes

\begin{align*}
    & \min{\underset{k=1, 2, \ldots, n}{\max}\vert p \sum\limits_{j=1}^m a_{k,j} - I_{des}\vert}\\
    & \min{\underset{k=1, 2, \ldots, n}{\max}\left( p \sum\limits_{j=1}^m a_{k,j} - I_{des}\right)^2} \\
    & \min{\underset{k=1, 2, \ldots, n}{\max}\left( p \mathbf{a}_k^\mathsf{T}\mathbf{1} - I_{des}\right)^2} \\
    & \min{\underset{k=1, 2, \ldots, n}{\max}\left( p^2 \mathbf{a}_k^\mathsf{T}\mathbf{a} - 2pI_{des} \mathbf{a}_k^\mathsf{T}\mathbf{1} + I_{des}^2 \right)} \\
    & \min{\underset{k=1, 2, \ldots, n}{\max}\left( p^2 \mathbf{a}_k^\mathsf{T}\mathbf{a} - 2pI_{des} \mathbf{a}_k^\mathsf{T}\mathbf{1} \right)}
    
\end{align*}

%lsmagic